### Plot the CCD dark image statistics

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, datashader as ds

from pathlib import Path
from astropy.io import fits
from scipy import stats
from datashader import transfer_functions as tf

In [ ]:
#set camera name
camera = "B0"

In [ ]:
cuts = pd.read_csv("cuts.txt", delim_whitespace = True)

In [ ]:
#generate file path
data_path = "../../"
file_path = data_path + "dark-stat-" + camera + ".fits"

In [ ]:
#read fits file into data frame
fitsfile = fits.open(file_path)
median_img = fitsfile["MEDIAN"].data
iqr_img = fitsfile["IQR"].data

#ravel data and fix byte order
median = median_img.ravel().newbyteorder().byteswap()
iqr = iqr_img.ravel().newbyteorder().byteswap()

In [ ]:
def fast_scatter(x,y,ax,x_range=None, y_range=None, **kwargs):
    if x_range == None:
        x_range=(np.min(x), np.max(x))
    if y_range == None:
        y_range=(np.min(y), np.max(y))
                 
    df = pd.DataFrame({"x":x, "y":y})
    cvs = ds.Canvas(plot_width = 700, plot_height = 700, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'x', 'y')
    ds.transfer_functions.Image.border=0
    img = tf.shade(agg, cmap = ["white", "black"])
    ax.imshow(img.to_masked_array(), origin="lower",cmap="binary", extent=(x_range[0],x_range[1],y_range[0], y_range[1]), interpolation=None)
    return ax

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
x_range = (np.min(median),0.05)
y_range = (0,0.04)
ax = fast_scatter(median, iqr,ax,x_range, y_range)
ax.set_xlabel("median (percentile)", size=20)
ax.set_ylabel("iqr (percentile)", size =20)
cuts_cam = cuts[cuts["camera"]==camera]
ax.axvline(cuts_cam["med_min"].to_numpy(),c="k", ls="--")
ax.axvline(cuts_cam["med_max"].to_numpy(), c="k", ls="--")
ax.axhline(cuts_cam["iqr_min"].to_numpy(),c="k", ls="--")
ax.axhline(cuts_cam["iqr_max"].to_numpy(),c="k", ls="--")

plt.ylim(-0.01,0.03)

#Uncomment if you want to plot axes in percentiles

# #convert ticks to percentile scale
# x_ticks = ax.get_xticks()
# x_percs = [np.round(stats.percentileofscore(median, p),4) for p in x_ticks]
# # ax.set_xticks(x_ticks)
# ax.set_xticklabels(x_percs)

# y_ticks = ax.get_yticks()
# y_percs = [np.round(stats.percentileofscore(median, p),4) for p in y_ticks]
# # ax.set_yticks(y_ticks)
# ax.set_yticklabels(y_percs)
# plt.title("Camera: B0", size=20)